V2: integrate the add event code into functions<br />
V3: get the portion of state while not record every state change

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
from math import factorial

In [13]:
# Hyper parameters
EPISODES = 1 # times every setting is replicated
# random.seed(1)
WriteFile = False
#WriteFile = True
FileAdd = '/Users/valarian/SJTU/SJTU/毕业论文/performanceA2M6HigherMu.csv'
WARMTIME = 500
RUNTIME = 50000

In [5]:
# calculate performance
'''
the indicators: 
serve:
average number of normal bikes: 
the customer lost rate
maintainance:
idle rate of fix servers
average BP bikes
average RC bikes
average DP bikes
'''

'\nthe indicators: \nserve:\naverage number of normal bikes: \nthe customer lost rate\nmaintainance:\nidle rate of fix servers\naverage BP bikes\naverage RC bikes\naverage DP bikes\n'

In [14]:
np.random.seed(1)
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Gamma: gathering rate
Mu: fix rate
Delta: distributing rate
RhoMtx: matrix of ride rates
N: number of fix servers
B_: valve value at broken pool
D_: valve value at distributing pool
'''

Instance = 4
FileAdd = 'C:\\Rebalancing\\nowModel\\result\\A2M6\V'+str(Instance)+'\\ODE\\'

np.random.seed(Instance)
A = 2
M = 6

def getPij(a):
    temp = np.log1p(np.random.rand(A,A))
    return (temp/sum(temp)).T
Pij = getPij(A)
ArrLst = np.log1p(np.random.rand(A))
Beta = 0.3
#RhoMtx = [[1.0] * A for i in range(A)]
#RhoMtx = np.random.rand(A, A)

Gamma = 2
Mu = 1.0
N = 1
Delta = 2
B_, D_ = 2, 2

# A = 2
# M = 6
# def getPij(a):
#     temp = np.random.rand(A,A)
#     return (temp/sum(temp)).T
# Pij = getPij(A)
# # Pij = [[0.3, 0.7],
# #        [0.7, 0.3]]
# # Pij = [[0.1, 0.2, 0.3, 0.4],
# #        [0.2, 0.3, 0.4, 0.1],
# #        [0.3, 0.4, 0.1, 0.2],
# #        [0.4, 0.3, 0.2, 0.1]]
# Beta = 0.3
# ArrLst = [5.0, 5.0]
# #ArrLst = [5.0, 5.0, 6.0, 7.0]
# Gamma = 1.0
# Mu = 1.0
# Delta = 1.0
RhoMtx = [[1.0, 0.5], 
          [0.5, 1.0]]
# RhoMtx = [[1.0] * A for i in range(A)]
# N = 1
# B_, D_ = 2, 2

In [46]:
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Gamma: gathering rate
Mu: fix rate
Delta: distributing rate
RhoMtx: matrix of ride rates
N: number of fix servers
B_: valve value at broken pool
D_: valve value at distributing pool
'''

A = 2
M = 6
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
# Pij = [[0.1, 0.2, 0.3, 0.4],
#        [0.2, 0.3, 0.4, 0.1],
#        [0.3, 0.4, 0.1, 0.2],
#        [0.4, 0.3, 0.2, 0.1]]
Beta = 0.4
ArrLst = [5.0, 5.0]
#ArrLst = [5.0, 5.0, 6.0, 7.0]
Gamma = 1.0
Mu = 1.0
Delta = 1.0
# RhoMtx = [[1.0, 1.0], 
#           [1.0, 1.0]]
RhoMtx = [[1.0] * A for i in range(A)]
N = 4
B_, D_ = 2, 2

%store -r cenState
%store -r cenPorState
State, portionState = cenState, cenPorState
for k,s in enumerate(State):
    State[s] = [0.0] * EPISODES

In [15]:
# simulate the central model
# with BP, RC, DP

class Model():
    '''
    This is the central model
    '''
    # initiate the parameters in this function
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_):     
        self.timeLimit = WARMTIME + RUNTIME
        self.areas = list(range(A))
        #self.epi = 0
        
        #self.Performance = [0] * EPISODES
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.B_, self.Gamma, self.Mu, self.N, self.Delta, self.D_ = \
            A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_
        
        def get_target_number():
            A, M, D_, ArrLst = self.A, self.M, self.D_, self.ArrLst
            arr = np.array(ArrLst)/sum(ArrLst)
            num_dis = [int(M*x) for x in arr]
            left = M - sum(num_dis)
            for i in np.argsort(ArrLst)[:left]:
                num_dis[i] += 1
            return num_dis
        self.num_dis = get_target_number()
        self.arr_rank = np.argsort(self.num_dis)[::-1]
        
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        
        # serve performance parameters
        self.normalBikes, self.brokenBikes = M, 0
        # maintainance performance parameters
        self.idleRate, self.BP, self.RC, self.DP = 0, 0, 0, 0
        self.arrivals, self.lostCustomers = 0, 0
        # indicators of time stamp
        [self.nt, self.bt, self.it, self.bpt, self.rct, self.dpt] = [0]*6
        self.enormalBikes, self.ebrokenBikes = 0, 0
        # maintainance performance parameters
        self.eidleRate, self.eBP, self.eRC, self.eDP = 0, 0, 0, 0
        self.Result = [0.0] * 7
        
        self.state1 = [int(M/A)]*A + [0]*3 # Nis, BP, FC, and DP
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        return self.state1, self.state2, self.T
        
    def setRecord(self, kind):
        if kind == 1:
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes - 1, self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP + 1, self.T
        elif kind == 2:
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it + (self.T - self.it)) / self.T 
                self.it = self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP - self.B_, self.T
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC + self.B_, self.T
        elif kind == 3:
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC - 1, self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP + 1, self.T
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it) / self.T
                self.it = self.T
        elif kind == 4: 
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes + self.D_, self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP - self.D_, self.T
        elif kind == -10:
            self.arrivals += 1
        elif kind == -11:
            self.lostCustomers += 1
        else: print('fucking wrong')
    
    def simulate(self):
        if WriteFile:
            with open(FileAdd, 'w') as fout:
                writer = csv.writer(fout)
                for i in range(EPISODES):
                    self.reset()
                    self.epi = i
                    while self.T <= self.timeLimit:
                        # print(self.T)
                        self.stepForward()
                writer.writerow([self.normalBikes, self.brokenBikes, self.lostCustomers/self.arrivals, self.idleRate, self.BP, self.RC, self.DP])  
        else:
            for i in range(EPISODES):
                self.epi = i
                self.reset()
                while self.T <= self.timeLimit:
                    self.stepForward()
                    #self.setRecord()
                print([self.enormalBikes, self.lostCustomers/self.arrivals, self.eidleRate, self.eBP, self.eRC, self.eDP])


    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(Gamma)
            next_time += self.T
            start, end = 'b', 'f'
            #print('add event 2')
        elif kind == 3:
            next_time = random.expovariate(Mu) 
            if self.state1[-2] < N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        else: 
            next_time = random.expovariate(Delta)
            next_time += self.T
            start, end = 'd', 'ni'
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        if random.random()<Beta:
            self.state1[-3] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] == B_:
                self.addEvent(2)
            self.setRecord(1)
        else:
            self.state1[self.terminal] += 1
            heapq.heappop(self.scheduler)
    def BPover(self):
        heapq.heappop(self.scheduler)
        for i in range(B_): 
            self.state1[-3] -= 1
            self.addEvent(3) 
            self.state1[-2] += 1
        if self.state1[-3] >= B_: self.addEvent(2)
        self.setRecord(2)
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[-2] <= N: heapq.heappop(self.F)
        self.state1[-2] -= 1
        self.state1[-1] += 1
        if self.state1[-1] == D_:
            self.addEvent(4)
        self.setRecord(3)
    def DPover(self):
#         heapq.heappop(self.scheduler)
#         self.state1[-1] -= D_
#         for i in range(A): self.state1[i] += D_/A
#         if self.state1[-1] >= D_: self.addEvent(4)
#         self.setRecord(4)
        heapq.heappop(self.scheduler)
        self.state1[-1] -= D_
        dbar = D_
        for i in self.arr_rank: 
            if self.state1[i]>=self.num_dis[i]: continue
            else: 
                alloc_n = min(self.num_dis[i]-self.state1[i], D_)
                self.state1[i] += alloc_n
                dbar -= alloc_n
                if dbar == 0: break
        if self.state1[-1] >= D_: self.addEvent(4)
        self.setRecord(4)
    def cusArr(self):
        #print(self.state1, self.state2)
        #print('------------------------')
        self.setRecord(-10)
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setRecord(-11)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: BP greater than B_
         3: a bike is fixed
         4: DP greater than D_
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.BPover() # 坏车达到阈值
        elif self.kind == 3:
            self.repair() # 修好一辆车
        elif self.kind == 4:
            self.DPover() # 再分配
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T


#B_ = 5
env = Model(A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_)
%time env.simulate()

[5.004774015383494, 0.31720927700348434, 0.8509303846621588, 0.6119164257170936, 0.23101935581698027, 0.1522242130794713]
CPU times: user 532 ms, sys: 6.6 ms, total: 538 ms
Wall time: 699 ms


In [44]:
names=['epi', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)
r[['a','b']] = r[['a','b']].astype(int)
#r.t = r.t/(RunTime+WarmTime)
r.head()

,epi,a,b,c,d,e,f,g,h,i,t
0,0,2,2,0,0,0,0,0,0,0,3.845590e-06
1,0,2,1,0,0,0,1,0,0,0,1.256588e-06
2,0,2,0,0,0,0,2,0,0,0,5.593862e-07
3,0,1,0,1,0,0,2,0,0,0,1.936154e-07
4,0,0,0,1,1,0,2,0,0,0,2.807333e-06


In [45]:
r.tail()

,epi,a,b,c,d,e,f,g,h,i,t
38675840,99,1,1,0,0,0,0,0,2,0,0.000002
38675841,99,0,1,0,1,0,0,0,2,0,0.000002
38675842,99,0,1,0,1,0,0,0,2,0,0.000002
38675843,99,0,0,0,1,1,0,0,2,0,0.000002
38675844,99,0,0,0,1,1,0,0,2,0,0.000004


In [53]:
re = []
for i in range(50):
    re.append(r[r.epi == i].drop(columns=['epi']).groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum())
re[5].t.sum()

1.0000029426798294

In [9]:
def getInterval(lst,s):
    mu, sigma = np.average(lst), np.var(lst)**0.5
    left95, right95 = mu-1.96*sigma, mu+1.96*sigma
    between = left95 <= portionState[s] <= right95
    if not between and mu!=0: print([s, mu, sigma, left95, right95, portionState[s], between])
    return [mu, sigma, left95, right95, portionState[s], between]

rateDict = {}
for s in State:
    rateDict[s] = getInterval(State[s],s)
#rateDict

In [19]:
rateDict

{(0, 0, 0, 0, 0, 0, 0, 0, 4): [0.017781354975568087,
  0.0071247704383448525,
  0.003816804916412177,
  0.031745905034724,
  0.018907079842857874,
  True],
 (0, 0, 0, 0, 0, 0, 0, 1, 3): [0.008621809326824931,
  0.0025262856051156534,
  0.00367028954079825,
  0.013573329112851612,
  0.009453539921428937,
  True],
 (0, 0, 0, 0, 0, 0, 0, 2, 2): [0.006917597924073688,
  0.001589639701054591,
  0.00380190411000669,
  0.010033291738140687,
  0.007090154941071703,
  True],
 (0, 0, 0, 0, 0, 0, 0, 3, 1): [0.003055163137068084,
  0.0014558993243928237,
  0.00020160046125814943,
  0.0059087258128780185,
  0.0032161621082682232,
  True],
 (0, 0, 0, 0, 0, 0, 0, 4, 0): [0.0008046599601620056,
  0.0006850284559775521,
  -0.0005379958135539966,
  0.0021473157338780077,
  0.0009730498641852203,
  True],
 (0, 0, 0, 0, 0, 0, 2, 0, 2): [0.022552319966347253,
  0.0046745931684989426,
  0.013390117356089326,
  0.03171452257660518,
  0.022586126272285616,
  True],
 (0, 0, 0, 0, 0, 0, 2, 1, 1): [0.00855301580

In [ ]:
print(np.average(rateDict[(0, 0, 4, 0, 0, 0, 0, 0, 0)]))
plt.plot(rateDict[(0, 0, 4, 0, 0, 0, 0, 0, 0)])

In [25]:
r = pd.read_csv(FileAdd,header = None)

r[(r.index<=80)&(r.index>=60)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
60,0.0,2.0,0,0,0,1,1,0,0,4.347103,-1,0,0,4.199329,[]
61,0.0,2.0,0,0,0,1,1,0,0,4.394023,-1,1,1,4.347103,[]
62,0.0,1.0,0,0,0,1,1,1,0,4.395580,-1,1,1,4.394023,[]
63,0.0,0.0,0,0,0,1,1,2,0,4.490974,-1,0,0,4.395580,[]
64,0.0,0.0,0,0,0,1,1,2,0,4.495842,1,0,1,4.490974,[]
65,0.0,1.0,0,0,0,1,0,2,0,4.555978,-1,1,1,4.495842,[]
66,0.0,0.0,0,0,0,1,0,3,0,4.558624,-1,0,0,4.555978,[]
67,0.0,0.0,0,0,0,1,0,3,0,4.706851,1,1,0,4.558624,[]
68,1.0,0.0,0,0,0,1,0,2,0,4.771952,1,0,0,4.706851,[]
69,2.0,0.0,0,0,0,0,0,2,0,4.809415,-1,0,0,4.771952,[]


In [119]:
names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)

r['sep'] = r.t.diff()
r = r[1:]
r[['a','b']] = r[['a','b']].astype(int)
r = r.drop(columns=['t'])
#r.head()
t = r.groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum()

t.sep = t.sep/RunTime
t.sort_values('sep', ascending=False)
t.head()

sep
a b c d e f g h i          
0 0 0 0 0 0 0 0 4  0.000118
              1 3  0.000547
              2 2  0.000874
              3 1  0.000775
              4 0  0.000247